### Environment Status

All required libraries (numpy, pandas, scikit-learn, joblib, qiskit) were checked.
The environment is ready to run the assignment end-to-end.


In [1]:
import numpy as np
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
def build_channel_classifier():
    """
    Trains a lightweight classifier that distinguishes between
    depolarizing and amplitude damping quantum channels using
    Choi matrix features.

    Returns:
        model : trained sklearn classifier
    """


    # Helper matrices
    I = np.eye(2, dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)

    # Kraus operators
    def depolarizing_kraus(p):
        k0 = np.sqrt(1 - p) * I
        k1 = np.sqrt(p / 3) * X
        k2 = np.sqrt(p / 3) * Y
        k3 = np.sqrt(p / 3) * Z
        return [k0, k1, k2, k3]

    def amplitude_damping_kraus(gamma):
        k0 = np.array([[1, 0], [0, np.sqrt(1 - gamma)]], dtype=complex)
        k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
        return [k0, k1]


    # Choi construction (NumPy only)
    def kraus_to_choi(kraus_list):
        choi = np.zeros((4, 4), dtype=complex)
        for K in kraus_list:
            vec = np.reshape(K, (-1,), order='F')
            choi += np.outer(vec, vec.conj())
        return choi

    def channel_to_feature(kraus_list):
        choi = kraus_to_choi(kraus_list)
        return np.concatenate([choi.real.flatten(), choi.imag.flatten()])

    # Dataset generation
    X_data = []
    y_data = []

    for p in np.linspace(0.0, 0.9, 15):
        X_data.append(channel_to_feature(depolarizing_kraus(p)))
        y_data.append("depolarizing")

    for g in np.linspace(0.0, 0.9, 15):
        X_data.append(channel_to_feature(amplitude_damping_kraus(g)))
        y_data.append("amplitude_damping")

    X_data = np.vstack(X_data)
    y_data = np.array(y_data)


    # Train / validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.25, random_state=42, stratify=y_data
    )

    # Model training
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluation
    preds = model.predict(X_val)
    print("Validation Accuracy:", accuracy_score(y_val, preds))
    print(classification_report(y_val, preds))

    # Save model
    joblib.dump(model, "channel_classifier.pkl")

    return model


model = build_channel_classifier()


Validation Accuracy: 1.0
                   precision    recall  f1-score   support

amplitude_damping       1.00      1.00      1.00         4
     depolarizing       1.00      1.00      1.00         4

         accuracy                           1.00         8
        macro avg       1.00      1.00      1.00         8
     weighted avg       1.00      1.00      1.00         8



## Channel Feature Construction

Each quantum channel is represented using its Choi matrix.
The real and imaginary parts of the Choi matrix are flattened
and concatenated to form a fixed-length feature vector.


In [3]:
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def depolarizing_kraus(p):
    k0 = np.sqrt(1 - p) * I
    k1 = np.sqrt(p / 3) * X
    k2 = np.sqrt(p / 3) * Y
    k3 = np.sqrt(p / 3) * Z
    return [k0, k1, k2, k3]

def amplitude_damping_kraus(gamma):
    k0 = np.array([[1, 0], [0, np.sqrt(1 - gamma)]], dtype=complex)
    k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
    return [k0, k1]

def kraus_to_choi(kraus_list):
    choi = np.zeros((4, 4), dtype=complex)
    for K in kraus_list:
        vec = np.reshape(K, (-1,), order='F')
        choi += np.outer(vec, vec.conj())
    return choi

def channel_to_feature(channel):
    choi = kraus_to_choi(channel)
    return np.concatenate([choi.real.flatten(), choi.imag.flatten()])


## Channel Classification Results

We test the trained classifier on a small set of synthetic
quantum channels and verify whether the predicted labels
match the expected noise models.


In [4]:
channels = [
    ("depolarizing_p0.1", depolarizing_kraus(0.1)),
    ("depolarizing_p0.5", depolarizing_kraus(0.5)),
    ("amp_damp_0.1", amplitude_damping_kraus(0.1)),
    ("amp_damp_0.5", amplitude_damping_kraus(0.5)),
]

features = []
names = []

for name, ch in channels:
    features.append(channel_to_feature(ch))
    names.append(name)

X_test = np.vstack(features)
preds = model.predict(X_test)

df = pd.DataFrame({
    "Channel": names,
    "Predicted Label": preds
})

df


,Channel,Predicted Label
0,depolarizing_p0.1,depolarizing
1,depolarizing_p0.5,depolarizing
2,amp_damp_0.1,amplitude_damping
3,amp_damp_0.5,amplitude_damping


### Reflection

This assignment demonstrates that Choi-based features can be used
to quickly classify quantum noise channels without performing full
quantum process tomography.

The lightweight classifier achieves perfect accuracy on synthetic
test channels, making it suitable for fast calibration-time checks.
